**Loading data and metadata**

In [ ]:
#  Illumina Infinium MethylationEPIC Beadchip data from 473 positive and 101 negative SARS-CoV-2 individuals 
GSE179325<-fread("path",header=T)  # check folder's readme to find data address
GSE179325 <- as.data.frame(GSE179325)
rownames(GSE179325) <- GSE179325$ID

# metadata list
metaGSE179325_list <- readLines("path") # check folder's readme to find data address
length(metaGSE179325_list)
# Severity inf: element 42 in list
metadata_string <- metaGSE179325_list[[42]]
metadata_split <- strsplit(metadata_string, "\t")[[1]]
disease_states <- metadata_split[-1]
disease_states <- gsub("disease state: ", "", disease_states)
disease_states <- trimws(disease_states)
table(disease_states)  # Check distribution of NEGATIVE, MILD, SEVERE
sample_info<-cbind(colnames(GSE179325[, -1]),disease_states)
colnames(sample_info)<- c("Sample_Name","Group")
sample_info<- as.data.frame(sample_info)

**Global methylation changes**

In [ ]:

# mean methylation per sample
mean_methylation_per_sample <- colMeans(GSE179325[, -1], na.rm = TRUE) 
sample_info$Mean_Methylation <- mean_methylation_per_sample[match(sample_info$Sample_Name, names(mean_methylation_per_sample))]

#  mean methylation per group
mean_methylation_per_group <- aggregate(Mean_Methylation ~ Group, data = sample_info, FUN = mean, na.rm=TRUE)
print(mean_methylation_per_group)

#  standard deviation per group 
sd_methylation_per_group <- aggregate(Mean_Methylation ~ Group, data = sample_info, FUN = sd, na.rm=TRUE)
print(sd_methylation_per_group)

# statistical testing 
kruskal.test(Mean_Methylation ~ Group, data = sample_info) 
pairwise.wilcox.test(sample_info$Mean_Methylation, sample_info$Group, p.adjust.method = "bonferroni") # Pairwise comparisons

**Visualisation**

In [ ]:
#  violin plot of global methylation
library(ggplot2)

ggplot(sample_info, aes(x = Group, y = Mean_Methylation, fill = Group)) + 
  geom_violin(trim = FALSE) + 
  geom_boxplot(width = 0.1, fill = "white") + 
  scale_fill_manual(values = c("lightblue", "lightgreen", "lightcoral")) + 
  labs(title = "Mean Methylation per Group (Violin Plot)", x = "Group", y = "Mean Methylation") +
  theme_bw() +  
  theme(legend.position = "none",
        axis.text = element_text(size = 12), 
        axis.title = element_text(size = 14), 
        title = element_text(size = 16))  